# Spark

[Spark](https://spark.apache.org/docs/1.6.2/index.html) (we are going to use version 1.6.2) is a fast and general-purpose cluster computing system, that allows to process large datasets using several machines. In the era of the *big data*, it has become a necessity to be able to distribute computing power and process data in parallel. We introduce here the basics of the framework.

In any case, you should have a look at the [official tutorial](https://spark.apache.org/docs/1.6.2/programming-guide.html).

## Hadoop File System (HDFS)

The main abstraction Spark offers is the *Resisilient Distributed Dataset* (RDD), which is a collection of elements distributed across the nodes of the cluster. The idea is that you typically **do not** load them in the memory, but rather process them *lazily*. They are initialized with files in the *Hadoop File System* (HDFS). 

Let us first see what there is in HDFS.

In [11]:
!hdfs dfs -ls

Found 1 items
-rw-r--r--   3 khau hadoop    2147813 2017-03-01 09:16 election-day-tweets.txt


*Note:* a line starting with `!` indicates a shell command. You can hence run this same command in a bash terminal.

We can now add a dataset to HFDS. In the `data` folder, there is text file containing 30000 tweets collected from the [Inauguration Day](https://en.wikipedia.org/wiki/United_States_presidential_inauguration) on January 20, 2017. 

In [12]:
!hdfs dfs -put data/election-day-tweets.txt

put: `election-day-tweets.txt': File exists


And check that the file is indeed in HDFS.

In [13]:
!hdfs dfs -ls

Found 1 items
-rw-r--r--   3 khau hadoop    2147813 2017-03-01 09:16 election-day-tweets.txt


*Note:* you can remove files from HDFS using `hdfs dfs -rm FILE`. 

## Dataset processing

We can now start playing with the tweets. We will instantiate the dataset from the text file.

In [16]:
from pyspark import SparkConf, SparkContext
conf = (SparkConf()
         .setMaster("local")
         .setAppName("My app"))
sc = SparkContext(conf = conf)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/hdp/2.5.3.0-37/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 577, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/hdp/2.5.3.0-37/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 690, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server

In [14]:
tweets = sc.textFile('data/election-day-tweets.txt')

NameError: name 'sc' is not defined

In [5]:
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)
print (distData)

ParallelCollectionRDD[2] at parallelize at PythonRDD.scala:423


The `tweets` variable is now an RDD. An RDD is *resilient* to faults by storing the sequence of operations applied to it so that it can easily recreate the state of the data structure in case of a problem. It can also be logically cut to be *distributed* on several machines.

The `sc` object is a special object, namely a [Spark Context](https://spark.apache.org/docs/1.6.2/api/python/pyspark.html#pyspark.SparkContext) obeject, that we configured to be available everywhere in your Python environment. It is the main interface with Spark that has numerous useful methods. Check the documentation!

Let us display a few tweets.

In [6]:
some_tweets = tweets.take(10)
for i, tweet in enumerate(some_tweets):
    print("%d: %s" % (i, tweet))

0: @Lawrence @HillaryClinton Two first  @SenSchumer tomorrow. @TheLastWord #brooklyn  TheRealAmerica #Vote #Democrats #NastyWomenVote #Senate
1: My @latimesopinion op-ed on historic #California #Senate race. First time an elected woman senator succeeds another.
2: https://t.co/cbjQTK0Q1V
3: #Senate Wisconsin Senate Preview: Johnson vs. Feingold
4: If Rubio Wins and #Trump Loses in #Florida... #HillaryClinton #Senate #RepublicanPrimary #Senaterace #Miami... https://t.co/zIeNEcVnMO
5: #Senate Wisconsin Senate Preview: Johnson vs. Feingold
6: bob day is an honest  person   #senate patterson . a loss to the senate
7: Make Republicans #PayAPrice!
8:  💙🇺🇸#VoteBLUE🔃theBallot🇺🇸💙
9:  #Congress #Senate #FlipItDem


Once created, the `tweets` dataset can be acted on by dataset **operations**. RDDs support two types of operations: *transformations*, which create a new RDD from an existing one, and *actions*, which return a value to the driver program after running a computation on the RDD. For instance, we can `count` the number of tweets. Is it a transofrmation or an action?

In [7]:
tweets.count()

30000

We can also `filter` the lines matching a specific string. Is it a transformation or an action ?

*Note*: `lower()` is a `str` method that converts the string to lower case. What happens if you do not do that? 

In [8]:
lines_with_trump = tweets.filter(lambda line: "trump" in line.lower())
lines_with_trump.count()

1023

In [9]:
lines_with_clinton = tweets.filter(lambda line : "clinton" in line.lower())
lines_with_clinton.count()

657

A typical processing pattern is [MapReduce](https://en.wikipedia.org/wiki/MapReduce), as popularized by Hadoop. The idea is that you apply a function to your dataset that filters or sorts it (**map**), and then combine the ouputs by computing something (**reduce**). The typical example is a word count. So, how many times each word occur among our tweets?  

In [10]:
# Split every line into words and flattens the result (aggregate them all together)
words = tweets.flatMap(lambda line: line.split())
words.take(15)

['@Lawrence',
 '@HillaryClinton',
 'Two',
 'first',
 '@SenSchumer',
 'tomorrow.',
 '@TheLastWord',
 '#brooklyn',
 'TheRealAmerica',
 '#Vote',
 '#Democrats',
 '#NastyWomenVote',
 '#Senate',
 'My',
 '@latimesopinion']

In [11]:
# Assign 1 to each word
counts = words.map(lambda word: (word, 1))
counts.take(10)

[('@Lawrence', 1),
 ('@HillaryClinton', 1),
 ('Two', 1),
 ('first', 1),
 ('@SenSchumer', 1),
 ('tomorrow.', 1),
 ('@TheLastWord', 1),
 ('#brooklyn', 1),
 ('TheRealAmerica', 1),
 ('#Vote', 1)]

In [12]:
# Combines the counts by adding up the value (1) of equal keys (words)
word_counts = counts.reduceByKey(lambda a, b: a + b)
word_counts.take(10)

[('https://t.co/3Eq1y14Gjr', 1),
 ('Gujarati', 2),
 ('@ptshrikant', 3),
 ('officials…', 1),
 ('@YourDreaDose', 1),
 ('#CLAD2016', 1),
 ('.@gastropoda', 1),
 ('Just', 130),
 ('BSP', 37),
 ('lagayye', 1)]

You should read the list of common [transformations](https://spark.apache.org/docs/1.6.2/programming-guide.html#transformations) and [actions](https://spark.apache.org/docs/1.6.2/programming-guide.html#actions) to get an idea of what you can do.

## Dataset statistics

Your turn! Try to extract some statisctis from the dataset:

1. Total number of words
2. Average number of words per tweet
3. Ten most frequent word

In [13]:
# Your code goes here

# 1. Total number of words
num_words = counts.values().sum()
print (num_words)

# 2. Average number of words per tweet
avg_word = num_words*1.0 / tweets.count()
print (avg_word)

# 3. Ten most frequent word

word_counts.takeOrdered(10, key=lambda x: -x[1])

300882
10.0294


[('Congress', 8526),
 ('the', 6494),
 ('to', 5829),
 ('of', 4654),
 ('for', 3964),
 ('in', 3794),
 ('congress', 3675),
 ('is', 3478),
 ('and', 3295),
 ('a', 2985)]

## Bigrams

A bigram is combination of two consecutive words. 

- Extract the ten most frequent bigrams from the dataset. 

Be careful not to consider the last word of a tweet and the first one of the next tweet!

*Hint:* note that the `map` and `flatMap` methods take a **function** as argument, meaning that you can define a function that you pass as argument:

```python
def process(line):
    # Do something
    return something

x = tweets.flatMap(process)
```

In [14]:
# Your code goes here
def process(line):
    result = [];
    words = line.split()
    for i in range(len(words) - 1):
        result.append((words[i],words[i+1]))
    return result 

x = tweets.flatMap(process)
print (x.take(10))

[('@Lawrence', '@HillaryClinton'), ('@HillaryClinton', 'Two'), ('Two', 'first'), ('first', '@SenSchumer'), ('@SenSchumer', 'tomorrow.'), ('tomorrow.', '@TheLastWord'), ('@TheLastWord', '#brooklyn'), ('#brooklyn', 'TheRealAmerica'), ('TheRealAmerica', '#Vote'), ('#Vote', '#Democrats')]
